In [1]:
import os
import requests
from bs4 import BeautifulSoup
import re
import csv
import glob as gl
import tqdm
import pandas as pd
import numpy as np

In [2]:
def prevailing_wind_direction(wind_dir_series):
    rad = np.deg2rad(wind_dir_series)
    sin_mean = np.nanmean(np.sin(rad))
    cos_mean = np.nanmean(np.cos(rad))
    mean_dir_rad = np.arctan2(sin_mean, cos_mean)
    mean_dir_deg = (np.rad2deg(mean_dir_rad)) % 360
    return mean_dir_deg

In [3]:
def download_file(file_url, download_dir):
    """Download the file from the specified URL."""
    local_filename = os.path.join(download_dir, os.path.basename(file_url))
    with requests.get(file_url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    print(f"Downloaded: {local_filename}")
    return os.path.basename(file_url)

def scrape_and_download(url, file_pattern, download_dir):
    """Scrape the website and download files matching the pattern."""
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all links on the webpage and download the matching files
    for link in soup.find_all('a', href=True):
        file_url = link['href']
        if file_pattern.match(file_url):
            # Prepend the base URL if necessary
            if not file_url.startswith('http'):
                file_url = url + file_url
            # Download the file
            download_file(file_url, download_dir)
            
def download_vito(year):
    # Directory where you want to save the downloaded files
    download_dir = '../../Data/marblepoint'
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    
    # Pattern to match the file names
    file_pattern = re.compile(r'mpt.*q1h\.txt')

    # Find data online
    url = 'https://amrc.ssec.wisc.edu/data/ftp/pub/aws/q1h/' + str(year) + '/'

    scrape_and_download(url, file_pattern, download_dir)

In [ ]:
for year in [i for i in range(1991, 2025)]:
    download_vito(year)

In [4]:
path = '../../Data/marblepoint/'
# create list of file names
aws_files = gl.glob(path + "/mpt*.txt")
# create data frame
columns = ['year', 'doy', 'month', 'day', 'hour', 'temperature', 'pressure', 'wind_speed', 'wind_direction', 'relative_humidity', '?']
content = []
for filename in tqdm.tqdm(aws_files, total=len(aws_files)):
    df = pd.read_csv(filename, names=columns, sep='\s+')
    df.drop(index=df.index[:2], axis=0, inplace=True)
    content.append(df)
vito = pd.concat(content)
# create datetime column and set as index:
vito['DateTime'] = vito['year'].astype(str) + '-' + vito['month'].astype(str) + '-' + vito['day'].astype(str) + ' ' + vito['hour'].astype(str)
vito['DateTime'] = pd.to_datetime(vito['DateTime'],format='%Y-%m-%d %H%M')
vito.index = vito['DateTime']
del vito['DateTime']
# make data into floats
for i in vito.columns:
    vito[i] = vito[i].astype(float)
# replace fill values with nans:
vito.replace(444.0, np.nan, inplace=True)
# resample to daily resolution
del vito['hour']
agg_dict = dict.fromkeys([col for col in vito.columns if col != 'wind_direction'], 'mean')
agg_dict['wind_direction'] = prevailing_wind_direction
vito = vito.resample('1D').agg(agg_dict)

vito

100%|███████████████████████████████████████████████████████████████████████████████| 402/402 [00:03<00:00, 101.01it/s]
C:\Users\Homo Naledi\AppData\Local\Temp\ipykernel_14640\2800808839.py:3: RuntimeWarning: Mean of empty slice
  sin_mean = np.nanmean(np.sin(rad))
C:\Users\Homo Naledi\AppData\Local\Temp\ipykernel_14640\2800808839.py:4: RuntimeWarning: Mean of empty slice
  cos_mean = np.nanmean(np.cos(rad))


,year,doy,month,day,temperature,pressure,wind_speed,relative_humidity,?,wind_direction
DateTime,,,,,,,,,,
1991-01-01,1991.0,1.0,1.0,1.0,1.757143,978.795238,2.109524,NaN,NaN,103.409802
1991-01-02,1991.0,2.0,1.0,2.0,-0.094737,981.172727,2.018182,NaN,NaN,101.445255
1991-01-03,1991.0,3.0,1.0,3.0,-0.895652,978.879167,1.429167,NaN,NaN,32.681130
1991-01-04,1991.0,4.0,1.0,4.0,-0.281818,975.629167,1.966667,NaN,NaN,343.478967
1991-01-05,1991.0,5.0,1.0,5.0,-1.713636,978.704167,3.095833,NaN,NaN,49.122511
...,...,...,...,...,...,...,...,...,...,...
2024-12-27,2024.0,362.0,12.0,27.0,-2.278261,974.737500,3.720833,NaN,NaN,12.713801
2024-12-28,2024.0,363.0,12.0,28.0,-3.217391,977.400000,3.733333,NaN,NaN,112.412724
2024-12-29,2024.0,364.0,12.0,29.0,-1.550000,977.254167,3.004167,NaN,NaN,351.762029


In [5]:
# Add years from Wang et al., 2022
t_column_name = 'Temperature(¡æ)'
p_column_name = 'Pressure(hPa)'
ws_column_name = 'Wind Speed(m/s)'
wd_column_name = 'Wind Direction'

TP = pd.read_csv('../../Data/marblepoint/Marble Point_3h.csv', header=0, encoding='latin1')
TP['DateTime'] = pd.to_datetime(dict(year=TP['Year'], month=TP['Month'], day=TP['Day'], hour=TP['Three-hourly observation time(UTC)']))
TP.index = TP['DateTime']
TP = TP[[t_column_name, p_column_name, ws_column_name, wd_column_name]]
TP.rename(columns={t_column_name: 'temperature', p_column_name: 'pressure', ws_column_name: 'wind_speed', wd_column_name: 'wind_direction'}, inplace=True)
agg_dict = dict.fromkeys([col for col in TP.columns if col != 'wind_direction'], 'mean')
agg_dict['wind_direction'] = prevailing_wind_direction
TP = TP.resample('1D').agg(agg_dict)

TP

C:\Users\Homo Naledi\AppData\Local\Temp\ipykernel_14640\2800808839.py:3: RuntimeWarning: Mean of empty slice
  sin_mean = np.nanmean(np.sin(rad))
C:\Users\Homo Naledi\AppData\Local\Temp\ipykernel_14640\2800808839.py:4: RuntimeWarning: Mean of empty slice
  cos_mean = np.nanmean(np.cos(rad))


,temperature,pressure,wind_speed,wind_direction
DateTime,,,,
1980-02-01,NaN,NaN,NaN,NaN
1980-02-02,NaN,NaN,NaN,NaN
1980-02-03,NaN,NaN,NaN,NaN
1980-02-04,NaN,NaN,NaN,NaN
1980-02-05,-7.740000,979.942857,3.071429,147.753647
...,...,...,...,...
2021-12-27,-4.257143,963.300000,4.757143,127.387010
2021-12-28,-2.960000,963.562500,2.600000,106.808682
2021-12-29,-4.375000,965.775000,5.785714,122.817198


In [6]:
df = pd.concat([vito, TP], ignore_index=False)
df = df[~df.index.duplicated(keep='first')]
df = df.sort_index()

df

,year,doy,month,day,temperature,pressure,wind_speed,relative_humidity,?,wind_direction
DateTime,,,,,,,,,,
1980-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-05,NaN,NaN,NaN,NaN,-7.740000,979.942857,3.071429,NaN,NaN,147.753647
...,...,...,...,...,...,...,...,...,...,...
2024-12-27,2024.0,362.0,12.0,27.0,-2.278261,974.737500,3.720833,NaN,NaN,12.713801
2024-12-28,2024.0,363.0,12.0,28.0,-3.217391,977.400000,3.733333,NaN,NaN,112.412724
2024-12-29,2024.0,364.0,12.0,29.0,-1.550000,977.254167,3.004167,NaN,NaN,351.762029


In [7]:
df.to_csv('../../Data/marblepoint/marblepoint.csv')